In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('GFM_data.csv', sep = '\t')

In [3]:
import tensorflow as tf
import string
import requests

In [4]:
text = df.get('Text').to_numpy()

In [5]:
import string
def clean_data(txt_elmt):
    # txt_elmt is each element (each post) in the "text" array
    tokens = txt_elmt.split() # each word
    
    # nothing changes. except removing punctuation
    mytable = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(mytable) for word in tokens]
    return [word.lower() for word in tokens if word.isalpha()]

#for i in text:
# try with text[0] first
tokens = clean_data(text[0]) # list of tokens
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
sufficiency = 200_000
for i in range(len(tokens)):
    sequence = tokens[i: i + length] # every 50-word
    lines += [' '. join(sequence)]
    if i > sufficiency:
        break

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
tokenizer = Tokenizer(oov_token="<OOV>") # OOV adds new words outside the trained words
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) # all words become numbers

In [9]:
# training and test sets
X = np.array([])
y = np.array([])
for i in sequences[:2]:
    #X += [i[:-1]]
    #y += [i[-1]]
    i = np.array([i[:-1]])
    print(type(i))
    print(i)
    X = np.append(X, i)
#X = np.array(X, dtype=object)
#y = np.array(y)
X

<class 'numpy.ndarray'>
[[121 120   4  20 119 118  21 117   9   5 116  19   8 115 114 113 112 111
    4  20 110 109  96 108  19  96 107 106   5 105 104 103 102   4  20 101
  100   7  18  99   6  17  16   5  98   3   4  97  15  95]]
<class 'numpy.ndarray'>
[[120   4  20 119 118  21 117   9   5 116  19   8 115 114 113 112 111   4
   20 110 109  96 108  19  96 107 106   5 105 104 103 102   4  20 101 100
    7  18  99   6  17  16   5  98   3   4  97  15  95  22]]


array([121., 120.,   4.,  20., 119., 118.,  21., 117.,   9.,   5., 116.,
        19.,   8., 115., 114., 113., 112., 111.,   4.,  20., 110., 109.,
        96., 108.,  19.,  96., 107., 106.,   5., 105., 104., 103., 102.,
         4.,  20., 101., 100.,   7.,  18.,  99.,   6.,  17.,  16.,   5.,
        98.,   3.,   4.,  97.,  15.,  95., 120.,   4.,  20., 119., 118.,
        21., 117.,   9.,   5., 116.,  19.,   8., 115., 114., 113., 112.,
       111.,   4.,  20., 110., 109.,  96., 108.,  19.,  96., 107., 106.,
         5., 105., 104., 103., 102.,   4.,  20., 101., 100.,   7.,  18.,
        99.,   6.,  17.,  16.,   5.,  98.,   3.,   4.,  97.,  15.,  95.,
        22.])

In [10]:
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
sequence_length = X[0].size

In [11]:
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax')) # we predict each word ==> use size = vocab_size, activation = softmax to get the prob for each predicted word

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 50)             6100      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 100)            60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 122)               12322     
Total params: 169,322
Trainable params: 169,322
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
train_y = np.asarray(y, dtype=object)
train_y = train_y.astype(np.float32)
train_y

array([], shape=(0, 122), dtype=float32)

In [15]:
train_X = np.asarray([X], dtype=object)[0]
train_X = train_X.astype(np.float32)

In [ ]:
model.fit(train_X, train_y, batch_size=256, epochs=100)

In [17]:
train_X = np.asarray([X], dtype=object)[0]
X

array([121., 120.,   4.,  20., 119., 118.,  21., 117.,   9.,   5., 116.,
        19.,   8., 115., 114., 113., 112., 111.,   4.,  20., 110., 109.,
        96., 108.,  19.,  96., 107., 106.,   5., 105., 104., 103., 102.,
         4.,  20., 101., 100.,   7.,  18.,  99.,   6.,  17.,  16.,   5.,
        98.,   3.,   4.,  97.,  15.,  95., 120.,   4.,  20., 119., 118.,
        21., 117.,   9.,   5., 116.,  19.,   8., 115., 114., 113., 112.,
       111.,   4.,  20., 110., 109.,  96., 108.,  19.,  96., 107., 106.,
         5., 105., 104., 103., 102.,   4.,  20., 101., 100.,   7.,  18.,
        99.,   6.,  17.,  16.,   5.,  98.,   3.,   4.,  97.,  15.,  95.,
        22.])